# Week 15 - Day 5: Pre-trained Models for Finance

## Learning Objectives
- Understand transfer learning concepts for financial applications
- Learn to use FinBERT for financial sentiment analysis
- Implement practical sentiment analysis on financial texts
- Extract and utilize embeddings for downstream tasks

---

## Table of Contents
1. [Transfer Learning for Finance](#1-transfer-learning-for-finance)
2. [FinBERT Introduction](#2-finbert-introduction)
3. [Sentiment Analysis Practical](#3-sentiment-analysis-practical)
4. [Using Embeddings](#4-using-embeddings)
5. [Trading Signal Generation](#5-trading-signal-generation)

---

In [ ]:
# Install required packages (uncomment if needed)
# !pip install transformers torch pandas numpy scikit-learn matplotlib seaborn yfinance

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    AutoModel,
    BertTokenizer,
    BertModel,
    pipeline
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

---

## 1. Transfer Learning for Finance

### What is Transfer Learning?

Transfer learning is a machine learning technique where a model trained on one task is reused as the starting point for a model on a different task. In NLP, this typically means:

1. **Pre-training**: A large language model is trained on massive amounts of text data
2. **Fine-tuning**: The pre-trained model is adapted to a specific downstream task

### Why Transfer Learning for Finance?

| Challenge | Solution via Transfer Learning |
|-----------|-------------------------------|
| Limited labeled financial data | Leverage knowledge from general text |
| Domain-specific vocabulary | Use finance-specific pre-trained models |
| Expensive annotation | Reduce need for large labeled datasets |
| Complex financial context | Utilize contextual embeddings |

### Pre-trained Models Landscape

```
General Models          Finance-Specific Models
├── BERT               ├── FinBERT
├── RoBERTa            ├── FinancialBERT
├── DistilBERT         ├── BloombergGPT
├── GPT-2/3            └── SEC-BERT
└── XLNet
```

In [ ]:
# Demonstrate the concept of transfer learning
# Compare general BERT vs FinBERT representations

class TransferLearningDemo:
    """
    Demonstrates why domain-specific pre-training matters.
    """
    
    def __init__(self):
        self.financial_terms = [
            "bull market",      # Positive market sentiment
            "bear market",      # Negative market sentiment  
            "puts",             # Options terminology
            "calls",            # Options terminology
            "short squeeze",    # Trading phenomenon
            "dividend yield",   # Investment metric
            "market cap",       # Company valuation
            "P/E ratio",        # Valuation metric
        ]
        
        self.financial_sentences = [
            "The Fed raised interest rates by 25 basis points.",
            "Apple reported earnings beat with strong iPhone sales.",
            "The yield curve inverted, signaling recession fears.",
            "Short sellers are covering their positions aggressively.",
            "The stock is trading at a significant discount to NAV.",
        ]
    
    def explain_transfer_learning(self):
        """Visual explanation of transfer learning concept."""
        
        print("="*70)
        print("TRANSFER LEARNING IN FINANCE")
        print("="*70)
        print()
        print("Stage 1: PRE-TRAINING (done by researchers)")
        print("-" * 50)
        print("📚 General BERT: Trained on Wikipedia + BookCorpus")
        print("   → Learns general language understanding")
        print()
        print("📈 FinBERT: Further trained on financial texts")
        print("   → Financial news, SEC filings, analyst reports")
        print("   → Learns financial domain knowledge")
        print()
        print("Stage 2: FINE-TUNING (what we do)")
        print("-" * 50)
        print("🎯 Task-specific adaptation:")
        print("   → Sentiment analysis")
        print("   → Named entity recognition")
        print("   → Question answering")
        print("   → Trading signal generation")
        print()
        print("="*70)
        
        return self.financial_terms, self.financial_sentences

# Run demonstration
demo = TransferLearningDemo()
terms, sentences = demo.explain_transfer_learning()

---

## 2. FinBERT Introduction

### What is FinBERT?

**FinBERT** is a BERT model pre-trained on financial communication text. There are several variants:

1. **ProsusAI/finbert** - Trained for financial sentiment analysis
2. **yiyanghkust/finbert-tone** - Trained on analyst reports
3. **ahmedrachid/FinancialBERT** - Trained on financial news

### Architecture

```
FinBERT Architecture:
┌─────────────────────────────────────┐
│         Classification Head         │  ← Sentiment: Positive/Negative/Neutral
├─────────────────────────────────────┤
│         [CLS] Token Output          │  ← Sentence representation
├─────────────────────────────────────┤
│      12 Transformer Layers          │  ← Contextual encoding
├─────────────────────────────────────┤
│         Token Embeddings            │  ← Word + Position + Segment
├─────────────────────────────────────┤
│           Input Text                │
└─────────────────────────────────────┘
```

### Key Parameters
- **Hidden size**: 768
- **Attention heads**: 12
- **Layers**: 12
- **Max sequence length**: 512 tokens

In [ ]:
# Load FinBERT model and tokenizer
print("Loading FinBERT model...")

# Using ProsusAI/finbert - most popular for sentiment analysis
MODEL_NAME = "ProsusAI/finbert"

# Load tokenizer and model
finbert_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
finbert_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
finbert_model.to(device)
finbert_model.eval()

print(f"✓ Model loaded: {MODEL_NAME}")
print(f"✓ Model parameters: {sum(p.numel() for p in finbert_model.parameters()):,}")
print(f"✓ Number of labels: {finbert_model.config.num_labels}")
print(f"✓ Label mapping: {finbert_model.config.id2label}")

In [ ]:
# Explore FinBERT architecture
def explore_model_architecture(model):
    """Display model architecture details."""
    
    print("="*70)
    print("FINBERT MODEL ARCHITECTURE")
    print("="*70)
    
    # Count parameters by layer type
    param_counts = {}
    for name, param in model.named_parameters():
        layer_type = name.split('.')[0]
        if layer_type not in param_counts:
            param_counts[layer_type] = 0
        param_counts[layer_type] += param.numel()
    
    print("\nParameter distribution:")
    print("-" * 40)
    for layer_type, count in param_counts.items():
        print(f"  {layer_type}: {count:,} parameters")
    
    print("\nModel configuration:")
    print("-" * 40)
    config = model.config
    print(f"  Hidden size: {config.hidden_size}")
    print(f"  Intermediate size: {config.intermediate_size}")
    print(f"  Attention heads: {config.num_attention_heads}")
    print(f"  Hidden layers: {config.num_hidden_layers}")
    print(f"  Vocabulary size: {config.vocab_size}")
    print(f"  Max position embeddings: {config.max_position_embeddings}")
    
    return param_counts

param_counts = explore_model_architecture(finbert_model)

In [ ]:
# Tokenization demonstration
def demonstrate_tokenization(tokenizer, texts):
    """Show how FinBERT tokenizes financial text."""
    
    print("="*70)
    print("TOKENIZATION EXAMPLES")
    print("="*70)
    
    for text in texts:
        print(f"\nOriginal: {text}")
        
        # Tokenize
        tokens = tokenizer.tokenize(text)
        token_ids = tokenizer.encode(text, add_special_tokens=True)
        
        print(f"Tokens: {tokens}")
        print(f"Token IDs: {token_ids}")
        print(f"Number of tokens: {len(tokens)}")
        print("-" * 50)

# Test tokenization
sample_texts = [
    "The stock surged 15% on earnings beat.",
    "Fed signals hawkish stance on inflation.",
    "Company announces $2B share buyback program."
]

demonstrate_tokenization(finbert_tokenizer, sample_texts)

---

## 3. Sentiment Analysis Practical

### Financial Sentiment Categories

| Sentiment | Description | Trading Signal |
|-----------|-------------|----------------|
| **Positive** | Bullish news, good earnings, upgrades | Long / Buy |
| **Negative** | Bearish news, misses, downgrades | Short / Sell |
| **Neutral** | Factual, no clear direction | Hold / No action |

### Use Cases in Trading
- News sentiment scoring
- Earnings call analysis
- Social media monitoring
- Analyst report parsing
- SEC filing analysis

In [ ]:
# Create financial news dataset
financial_news = pd.DataFrame({
    'headline': [
        # Positive headlines
        "Apple reports record quarterly revenue, beating analyst expectations",
        "Tesla shares surge after better-than-expected delivery numbers",
        "Microsoft announces $60 billion share buyback program",
        "Amazon Web Services growth accelerates, driving stock higher",
        "Goldman Sachs upgrades NVIDIA to buy on AI demand",
        "JP Morgan reports strong trading revenue amid market volatility",
        "Retail sales rise more than expected, boosting consumer stocks",
        "Biotech firm announces breakthrough cancer treatment results",
        
        # Negative headlines
        "Bank of America faces $500 million fine over regulatory violations",
        "Meta shares plunge on weak advertising revenue guidance",
        "Boeing reports wider-than-expected quarterly loss",
        "Intel warns of continued chip demand weakness",
        "Retail giant announces 10,000 layoffs amid slowing sales",
        "Oil prices crash on recession fears and demand concerns",
        "Credit Suisse shares hit record low on banking crisis fears",
        "Pharmaceutical company recalls popular drug over safety concerns",
        
        # Neutral headlines
        "Federal Reserve holds interest rates steady as expected",
        "Company completes previously announced acquisition",
        "CEO to present at upcoming investor conference",
        "Board of directors appoints new chief financial officer",
        "Company releases annual sustainability report",
        "Stock split takes effect at market open today",
        "Quarterly dividend declared at unchanged rate",
        "Company announces date for next earnings release",
    ],
    'expected_sentiment': [
        'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive',
        'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative',
        'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
    ]
})

print(f"Created dataset with {len(financial_news)} headlines")
print(f"\nSentiment distribution:")
print(financial_news['expected_sentiment'].value_counts())

In [ ]:
# FinBERT Sentiment Analyzer
class FinBERTSentimentAnalyzer:
    """
    Financial sentiment analyzer using FinBERT.
    """
    
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.labels = ['positive', 'negative', 'neutral']
    
    def analyze_single(self, text):
        """
        Analyze sentiment of a single text.
        
        Returns:
            dict with sentiment label and probabilities
        """
        # Tokenize
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=512,
            padding=True
        ).to(self.device)
        
        # Get predictions
        with torch.no_grad():
            outputs = self.model(**inputs)
            probs = F.softmax(outputs.logits, dim=-1)
        
        # Get results
        probs_np = probs.cpu().numpy()[0]
        pred_idx = np.argmax(probs_np)
        
        return {
            'text': text,
            'sentiment': self.labels[pred_idx],
            'confidence': float(probs_np[pred_idx]),
            'probabilities': {
                label: float(prob) 
                for label, prob in zip(self.labels, probs_np)
            }
        }
    
    def analyze_batch(self, texts, batch_size=8):
        """
        Analyze sentiment of multiple texts.
        
        Returns:
            DataFrame with sentiment results
        """
        results = []
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            # Tokenize batch
            inputs = self.tokenizer(
                batch_texts,
                return_tensors='pt',
                truncation=True,
                max_length=512,
                padding=True
            ).to(self.device)
            
            # Get predictions
            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = F.softmax(outputs.logits, dim=-1)
            
            # Process results
            probs_np = probs.cpu().numpy()
            
            for j, text in enumerate(batch_texts):
                pred_idx = np.argmax(probs_np[j])
                results.append({
                    'text': text,
                    'sentiment': self.labels[pred_idx],
                    'confidence': float(probs_np[j][pred_idx]),
                    'positive_prob': float(probs_np[j][0]),
                    'negative_prob': float(probs_np[j][1]),
                    'neutral_prob': float(probs_np[j][2]),
                })
        
        return pd.DataFrame(results)
    
    def get_sentiment_score(self, text):
        """
        Get a continuous sentiment score (-1 to 1).
        Useful for trading signals.
        """
        result = self.analyze_single(text)
        probs = result['probabilities']
        
        # Score: positive - negative (neutral doesn't affect)
        score = probs['positive'] - probs['negative']
        
        return score

# Initialize analyzer
analyzer = FinBERTSentimentAnalyzer(finbert_model, finbert_tokenizer, device)
print("✓ FinBERT Sentiment Analyzer initialized")

In [ ]:
# Analyze financial news sentiment
print("Analyzing sentiment for financial headlines...\n")

# Analyze all headlines
results_df = analyzer.analyze_batch(financial_news['headline'].tolist())

# Add expected sentiment for comparison
results_df['expected'] = financial_news['expected_sentiment'].values
results_df['correct'] = results_df['sentiment'] == results_df['expected']

# Display results
print("Sample Results:")
print("="*100)
for _, row in results_df.head(6).iterrows():
    status = "✓" if row['correct'] else "✗"
    print(f"{status} [{row['sentiment'].upper():8}] (conf: {row['confidence']:.2f}) {row['text'][:60]}...")

print("\n" + "="*100)
accuracy = results_df['correct'].mean()
print(f"\nOverall Accuracy: {accuracy:.1%}")

In [ ]:
# Visualize sentiment analysis results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Confusion Matrix
ax1 = axes[0, 0]
cm = confusion_matrix(
    results_df['expected'], 
    results_df['sentiment'],
    labels=['positive', 'negative', 'neutral']
)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=['Positive', 'Negative', 'Neutral'],
            yticklabels=['Positive', 'Negative', 'Neutral'])
ax1.set_xlabel('Predicted')
ax1.set_ylabel('Actual')
ax1.set_title('Confusion Matrix')

# 2. Confidence Distribution
ax2 = axes[0, 1]
for sentiment in ['positive', 'negative', 'neutral']:
    mask = results_df['sentiment'] == sentiment
    ax2.hist(results_df.loc[mask, 'confidence'], alpha=0.6, 
             label=sentiment.capitalize(), bins=15)
ax2.set_xlabel('Confidence')
ax2.set_ylabel('Frequency')
ax2.set_title('Prediction Confidence Distribution')
ax2.legend()

# 3. Probability Distribution by Sentiment
ax3 = axes[1, 0]
prob_data = results_df.groupby('sentiment')[['positive_prob', 'negative_prob', 'neutral_prob']].mean()
prob_data.plot(kind='bar', ax=ax3)
ax3.set_xlabel('Predicted Sentiment')
ax3.set_ylabel('Average Probability')
ax3.set_title('Average Probability Distribution')
ax3.legend(['Positive', 'Negative', 'Neutral'])
ax3.tick_params(axis='x', rotation=0)

# 4. Sentiment Score Distribution
ax4 = axes[1, 1]
results_df['sentiment_score'] = results_df['positive_prob'] - results_df['negative_prob']
colors = ['green' if x > 0.2 else 'red' if x < -0.2 else 'gray' 
          for x in results_df['sentiment_score']]
ax4.bar(range(len(results_df)), results_df['sentiment_score'], color=colors, alpha=0.7)
ax4.axhline(y=0.2, color='green', linestyle='--', alpha=0.5, label='Buy threshold')
ax4.axhline(y=-0.2, color='red', linestyle='--', alpha=0.5, label='Sell threshold')
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax4.set_xlabel('Headline Index')
ax4.set_ylabel('Sentiment Score')
ax4.set_title('Sentiment Scores (Positive - Negative)')
ax4.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Interactive sentiment analysis
def analyze_custom_text(text):
    """Analyze custom financial text."""
    result = analyzer.analyze_single(text)
    
    print("="*70)
    print("SENTIMENT ANALYSIS RESULT")
    print("="*70)
    print(f"\nText: {text}\n")
    print(f"Sentiment: {result['sentiment'].upper()}")
    print(f"Confidence: {result['confidence']:.1%}\n")
    print("Probability Distribution:")
    
    # Visual bar
    for label, prob in result['probabilities'].items():
        bar = '█' * int(prob * 40)
        print(f"  {label.capitalize():10} [{bar:40}] {prob:.1%}")
    
    # Trading signal
    score = result['probabilities']['positive'] - result['probabilities']['negative']
    print(f"\n📊 Sentiment Score: {score:+.3f}")
    
    if score > 0.3:
        print("📈 Trading Signal: STRONG BUY")
    elif score > 0.1:
        print("📈 Trading Signal: BUY")
    elif score < -0.3:
        print("📉 Trading Signal: STRONG SELL")
    elif score < -0.1:
        print("📉 Trading Signal: SELL")
    else:
        print("➡️  Trading Signal: HOLD")
    
    return result

# Test with various headlines
test_headlines = [
    "Company posts record profits as demand exceeds expectations",
    "SEC launches investigation into accounting irregularities",
    "Quarterly results in line with analyst estimates",
]

for headline in test_headlines:
    _ = analyze_custom_text(headline)
    print()

---

## 4. Using Embeddings

### What are Embeddings?

Embeddings are dense vector representations of text that capture semantic meaning. They can be used for:

1. **Similarity search** - Finding similar documents/news
2. **Clustering** - Grouping related content
3. **Feature extraction** - Input to downstream models
4. **Visualization** - Understanding text relationships

### Extraction Methods

| Method | Description | Use Case |
|--------|-------------|----------|
| **[CLS] Token** | First token output | Classification tasks |
| **Mean Pooling** | Average of all tokens | General purpose |
| **Max Pooling** | Max across tokens | Capturing key features |

In [ ]:
# Load FinBERT base model for embeddings (without classification head)
print("Loading FinBERT base model for embeddings...")

# Use base model (without classifier) for embeddings
finbert_base = AutoModel.from_pretrained(MODEL_NAME)
finbert_base.to(device)
finbert_base.eval()

print(f"✓ Base model loaded")
print(f"✓ Hidden size: {finbert_base.config.hidden_size}")

In [ ]:
# Embedding Extractor Class
class FinBERTEmbedder:
    """
    Extract embeddings from FinBERT for financial texts.
    """
    
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
    
    def get_embeddings(self, texts, pooling='cls', batch_size=8):
        """
        Extract embeddings from texts.
        
        Args:
            texts: List of texts
            pooling: 'cls', 'mean', or 'max'
            batch_size: Batch size for processing
            
        Returns:
            numpy array of embeddings (n_texts, hidden_size)
        """
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            # Tokenize
            inputs = self.tokenizer(
                batch_texts,
                return_tensors='pt',
                truncation=True,
                max_length=512,
                padding=True
            ).to(self.device)
            
            # Get embeddings
            with torch.no_grad():
                outputs = self.model(**inputs)
                hidden_states = outputs.last_hidden_state
                attention_mask = inputs['attention_mask']
            
            # Apply pooling
            if pooling == 'cls':
                embeddings = hidden_states[:, 0, :]  # [CLS] token
            elif pooling == 'mean':
                # Mean pooling with attention mask
                mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
                sum_embeddings = torch.sum(hidden_states * mask_expanded, 1)
                sum_mask = mask_expanded.sum(1)
                embeddings = sum_embeddings / sum_mask
            elif pooling == 'max':
                # Max pooling
                mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
                hidden_states[mask_expanded == 0] = -1e9
                embeddings = torch.max(hidden_states, 1)[0]
            else:
                raise ValueError(f"Unknown pooling method: {pooling}")
            
            all_embeddings.append(embeddings.cpu().numpy())
        
        return np.vstack(all_embeddings)
    
    def compute_similarity(self, text1, text2):
        """
        Compute cosine similarity between two texts.
        """
        emb1 = self.get_embeddings([text1])[0]
        emb2 = self.get_embeddings([text2])[0]
        
        # Cosine similarity
        similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
        
        return similarity
    
    def find_similar(self, query, corpus, top_k=5):
        """
        Find most similar texts in corpus.
        """
        # Get embeddings
        query_emb = self.get_embeddings([query])[0]
        corpus_embs = self.get_embeddings(corpus)
        
        # Compute similarities
        similarities = np.dot(corpus_embs, query_emb) / (
            np.linalg.norm(corpus_embs, axis=1) * np.linalg.norm(query_emb)
        )
        
        # Get top-k
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        return [(corpus[i], similarities[i]) for i in top_indices]

# Initialize embedder
embedder = FinBERTEmbedder(finbert_base, finbert_tokenizer, device)
print("✓ FinBERT Embedder initialized")

In [ ]:
# Demonstrate embedding extraction
print("Extracting embeddings...\n")

# Get embeddings for all headlines
headlines = financial_news['headline'].tolist()
embeddings = embedder.get_embeddings(headlines, pooling='cls')

print(f"Embeddings shape: {embeddings.shape}")
print(f"  - {embeddings.shape[0]} headlines")
print(f"  - {embeddings.shape[1]} dimensions per embedding")

# Show embedding statistics
print(f"\nEmbedding statistics:")
print(f"  Mean: {embeddings.mean():.4f}")
print(f"  Std: {embeddings.std():.4f}")
print(f"  Min: {embeddings.min():.4f}")
print(f"  Max: {embeddings.max():.4f}")

In [ ]:
# Similarity search demonstration
print("="*70)
print("SEMANTIC SIMILARITY SEARCH")
print("="*70)

# Query
query = "Technology company reports strong quarterly earnings"
print(f"\nQuery: {query}\n")
print("Most similar headlines:")
print("-" * 70)

similar = embedder.find_similar(query, headlines, top_k=5)
for i, (text, score) in enumerate(similar, 1):
    print(f"{i}. (similarity: {score:.3f}) {text}")

In [ ]:
# Compare similarity between pairs
print("="*70)
print("PAIRWISE SIMILARITY COMPARISON")
print("="*70)

pairs = [
    ("Stock price rises on earnings beat", "Company reports better than expected profits"),
    ("Stock price rises on earnings beat", "Weather forecast predicts rain tomorrow"),
    ("Fed raises interest rates", "Central bank tightens monetary policy"),
    ("Fed raises interest rates", "Apple launches new iPhone"),
]

for text1, text2 in pairs:
    sim = embedder.compute_similarity(text1, text2)
    print(f"\nSimilarity: {sim:.3f}")
    print(f"  Text 1: {text1}")
    print(f"  Text 2: {text2}")

In [ ]:
# Visualize embeddings with t-SNE
print("Creating t-SNE visualization...")

# Reduce dimensions
tsne = TSNE(n_components=2, random_state=42, perplexity=8)
embeddings_2d = tsne.fit_transform(embeddings)

# Create visualization
fig, ax = plt.subplots(figsize=(12, 8))

# Color by sentiment
colors = {'positive': 'green', 'negative': 'red', 'neutral': 'blue'}
sentiments = financial_news['expected_sentiment'].values

for sentiment in ['positive', 'negative', 'neutral']:
    mask = sentiments == sentiment
    ax.scatter(
        embeddings_2d[mask, 0],
        embeddings_2d[mask, 1],
        c=colors[sentiment],
        label=sentiment.capitalize(),
        alpha=0.7,
        s=100
    )

# Add some labels
for i in range(0, len(headlines), 3):
    ax.annotate(
        headlines[i][:25] + "...",
        (embeddings_2d[i, 0], embeddings_2d[i, 1]),
        fontsize=8,
        alpha=0.7
    )

ax.set_xlabel('t-SNE Dimension 1')
ax.set_ylabel('t-SNE Dimension 2')
ax.set_title('Financial Headlines Embeddings (t-SNE Visualization)')
ax.legend()
plt.tight_layout()
plt.show()

print("✓ Notice how headlines cluster by sentiment!")

In [ ]:
# Using embeddings as features for classification
print("="*70)
print("EMBEDDINGS AS FEATURES FOR CLASSIFICATION")
print("="*70)

# Prepare data
X = embeddings
y = financial_news['expected_sentiment'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print(f"\nTraining samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

# Train logistic regression on embeddings
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)

print(f"\nTraining accuracy: {train_acc:.1%}")
print(f"Testing accuracy: {test_acc:.1%}")

print("\nClassification Report:")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

---

## 5. Trading Signal Generation

### From Sentiment to Signals

Converting sentiment analysis into actionable trading signals:

```
News/Text → Sentiment Analysis → Signal Score → Trading Decision
                    ↓                  ↓               ↓
              FinBERT          -1 to +1 scale    Buy/Sell/Hold
```

### Signal Aggregation Methods

| Method | Formula | Use Case |
|--------|---------|----------|
| Simple Average | $\bar{s} = \frac{1}{n}\sum_{i=1}^{n} s_i$ | Equal weighting |
| Time-Weighted | $s_t = \sum_{i} w_i \cdot s_i$ where $w_i \propto e^{-\lambda \cdot age_i}$ | Recent news matters more |
| Confidence-Weighted | $s_c = \sum_{i} c_i \cdot s_i / \sum c_i$ | Trust confident predictions |

In [ ]:
# Trading Signal Generator
class SentimentTradingSignal:
    """
    Generate trading signals from sentiment analysis.
    """
    
    def __init__(self, analyzer):
        self.analyzer = analyzer
    
    def compute_sentiment_score(self, text):
        """
        Compute sentiment score for single text.
        Returns score between -1 (very negative) and +1 (very positive).
        """
        result = self.analyzer.analyze_single(text)
        probs = result['probabilities']
        
        # Weighted score: positive - negative, scaled by confidence
        score = probs['positive'] - probs['negative']
        
        return {
            'text': text,
            'score': score,
            'confidence': result['confidence'],
            'sentiment': result['sentiment']
        }
    
    def aggregate_signals(self, texts, method='simple'):
        """
        Aggregate multiple sentiment scores into single signal.
        
        Args:
            texts: List of (text, weight) tuples or just texts
            method: 'simple', 'confidence_weighted'
        """
        scores = []
        confidences = []
        
        for text in texts:
            result = self.compute_sentiment_score(text)
            scores.append(result['score'])
            confidences.append(result['confidence'])
        
        if method == 'simple':
            agg_score = np.mean(scores)
        elif method == 'confidence_weighted':
            agg_score = np.average(scores, weights=confidences)
        else:
            raise ValueError(f"Unknown method: {method}")
        
        return {
            'aggregate_score': agg_score,
            'individual_scores': scores,
            'confidences': confidences,
            'n_articles': len(texts)
        }
    
    def generate_signal(self, score, thresholds=None):
        """
        Generate trading signal from score.
        
        Args:
            score: Sentiment score (-1 to 1)
            thresholds: Dict with 'strong_buy', 'buy', 'sell', 'strong_sell'
        """
        if thresholds is None:
            thresholds = {
                'strong_buy': 0.5,
                'buy': 0.2,
                'sell': -0.2,
                'strong_sell': -0.5
            }
        
        if score >= thresholds['strong_buy']:
            return 'STRONG_BUY', 1.0
        elif score >= thresholds['buy']:
            return 'BUY', 0.5
        elif score <= thresholds['strong_sell']:
            return 'STRONG_SELL', -1.0
        elif score <= thresholds['sell']:
            return 'SELL', -0.5
        else:
            return 'HOLD', 0.0

# Initialize signal generator
signal_generator = SentimentTradingSignal(analyzer)
print("✓ Trading Signal Generator initialized")

In [ ]:
# Simulate trading signal generation for a stock
print("="*70)
print("TRADING SIGNAL SIMULATION: Apple Inc. (AAPL)")
print("="*70)

# Simulated news feed for Apple
apple_news = [
    "Apple reports record iPhone 15 Pro sales exceeding analyst estimates",
    "Apple's services revenue continues strong growth trajectory",
    "Warren Buffett increases Berkshire's stake in Apple",
    "Apple announces expansion of AI features in upcoming iOS update",
    "Analysts raise Apple price targets after strong guidance",
    "Minor supply chain disruption reported for Apple Watch production",
]

# Analyze each headline
print("\nIndividual Article Analysis:")
print("-" * 70)
for i, news in enumerate(apple_news, 1):
    result = signal_generator.compute_sentiment_score(news)
    signal, _ = signal_generator.generate_signal(result['score'])
    print(f"{i}. Score: {result['score']:+.3f} | {result['sentiment']:8} | {news[:50]}...")

# Aggregate signal
print("\n" + "="*70)
print("AGGREGATED SIGNAL")
print("="*70)

agg_result = signal_generator.aggregate_signals(apple_news, method='confidence_weighted')
final_signal, position_size = signal_generator.generate_signal(agg_result['aggregate_score'])

print(f"\nAggregate Sentiment Score: {agg_result['aggregate_score']:+.3f}")
print(f"Number of Articles Analyzed: {agg_result['n_articles']}")
print(f"\n🎯 TRADING SIGNAL: {final_signal}")
print(f"📊 Suggested Position Size: {abs(position_size)*100:.0f}%")

In [ ]:
# Visualize signal generation process
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Individual scores
ax1 = axes[0]
scores = agg_result['individual_scores']
colors = ['green' if s > 0 else 'red' for s in scores]
bars = ax1.barh(range(len(scores)), scores, color=colors, alpha=0.7)
ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax1.axvline(x=0.2, color='green', linestyle='--', alpha=0.5)
ax1.axvline(x=-0.2, color='red', linestyle='--', alpha=0.5)
ax1.set_yticks(range(len(scores)))
ax1.set_yticklabels([f"Article {i+1}" for i in range(len(scores))])
ax1.set_xlabel('Sentiment Score')
ax1.set_title('Individual Article Sentiment Scores')
ax1.set_xlim(-1, 1)

# Signal gauge
ax2 = axes[1]
agg_score = agg_result['aggregate_score']

# Create gauge-like visualization
theta = np.linspace(0, np.pi, 100)
r = 1

# Background arcs
ax2.fill_between(np.cos(theta[:25]), 0, np.sin(theta[:25]), alpha=0.3, color='darkred')
ax2.fill_between(np.cos(theta[25:40]), 0, np.sin(theta[25:40]), alpha=0.3, color='red')
ax2.fill_between(np.cos(theta[40:60]), 0, np.sin(theta[40:60]), alpha=0.3, color='gray')
ax2.fill_between(np.cos(theta[60:75]), 0, np.sin(theta[60:75]), alpha=0.3, color='green')
ax2.fill_between(np.cos(theta[75:]), 0, np.sin(theta[75:]), alpha=0.3, color='darkgreen')

# Needle
needle_angle = np.pi/2 + (agg_score * np.pi/2)
ax2.arrow(0, 0, 0.8*np.cos(needle_angle), 0.8*np.sin(needle_angle),
          head_width=0.1, head_length=0.1, fc='black', ec='black')

# Labels
ax2.text(-0.9, -0.2, 'Strong Sell', fontsize=10)
ax2.text(-0.5, 0.7, 'Sell', fontsize=10)
ax2.text(-0.1, 0.9, 'Hold', fontsize=10)
ax2.text(0.4, 0.7, 'Buy', fontsize=10)
ax2.text(0.7, -0.2, 'Strong Buy', fontsize=10)

ax2.text(0, -0.4, f'Score: {agg_score:+.3f}\nSignal: {final_signal}', 
         ha='center', fontsize=12, fontweight='bold')

ax2.set_xlim(-1.2, 1.2)
ax2.set_ylim(-0.6, 1.2)
ax2.set_aspect('equal')
ax2.axis('off')
ax2.set_title('Aggregate Trading Signal', fontsize=12)

plt.tight_layout()
plt.show()

---

## 6. Exercises

### Exercise 1: Fine-tune FinBERT
Create a custom dataset and fine-tune FinBERT for a specific task (e.g., earnings call sentiment).

### Exercise 2: Multi-stock Sentiment Dashboard
Build a system that tracks sentiment for multiple stocks simultaneously.

### Exercise 3: Sentiment-Based Strategy
Backtest a trading strategy that uses sentiment signals as entry/exit triggers.

### Exercise 4: Compare Models
Compare FinBERT with other models (BERT, RoBERTa) on financial sentiment tasks.

---

## Summary

### Key Takeaways

1. **Transfer Learning** enables leveraging large pre-trained models for finance-specific tasks

2. **FinBERT** is specifically trained on financial text and understands domain vocabulary

3. **Sentiment Analysis** can be directly applied to:
   - News headlines
   - Earnings reports
   - Social media
   - Analyst reports

4. **Embeddings** are versatile features that can be used for:
   - Similarity search
   - Clustering
   - Classification
   - Visualization

5. **Trading Signals** require careful aggregation and threshold tuning

### Next Steps
- Day 6: Building Custom Transformers for Finance
- Integrate sentiment signals with price data
- Build production-ready sentiment pipelines

In [ ]:
# Quick reference - Using FinBERT pipeline (simplest approach)
print("QUICK REFERENCE: Using HuggingFace Pipeline")
print("="*70)

# Create sentiment pipeline
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    device=0 if torch.cuda.is_available() else -1
)

# Quick analysis
test_texts = [
    "Revenue growth exceeded expectations with strong margins.",
    "Company faces regulatory investigation over accounting practices.",
    "Board approves quarterly dividend at unchanged rate."
]

print("\nResults:")
for text, result in zip(test_texts, sentiment_pipeline(test_texts)):
    print(f"  [{result['label']:8}] ({result['score']:.2f}) {text[:50]}...")

print("\n✓ Pipeline is the quickest way to use pre-trained models!")